<H1 align="center"> 🏒 Project: NHL 🏒 </H1>

Data for this project can be found in [this](https://www.kaggle.com/datasets/martinellis/nhl-game-data) kaggle link. According to the author:

> The data represents all the official metrics measured for each game in the NHL (from 200 to 2019). I intend to update it semi-regularly depending on development progress of my database server.

We loaded the data into a SQLite database using the simple [DB Browser for SQLite](https://sqlitebrowser.org/).

## Data exploration with SQL and Plotly

Using our SQLite database we will answer exploratory questions using SQL queries. To this end, we'll use python's `sqlite3` module to run queries and `pandas` to get our results in dataframe objects.

The schema for this database is given in the figure bellow

![DB Schema](table_relationships.JPG)

For our visualizations we'll use Plotly for its awesome interactive functions!

In [5]:
import sqlite3 as sql
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [6]:
import plotly.io as pio
pio.templates.default = "simple_white"

# Connecting to Database

In [7]:
con = sql.connect("nhl-data.db")
cur = con.cursor()

# Total number of goals (and games) per season

In [8]:
# Number of goals and games per season
query = """
SELECT
    SUBSTR(season, 1, 4) AS season_start,
    SUM(away_goals) + SUM(home_goals) AS total_goals,
    COUNT(game_id) AS number_of_games,
    CAST(SUM(away_goals) + SUM(home_goals) AS REAL) / COUNT(game_id) AS goals_per_game
FROM
    game
GROUP BY
    season_start
ORDER BY
    season_start;
"""
df_goals_season = pd.read_sql(query, con)

In [9]:
df_goals_season

,season_start,total_goals,number_of_games,goals_per_game
0,2000,6782,1230,5.513821
1,2001,6442,1230,5.237398
2,2002,6530,1230,5.308943
3,2003,6318,1230,5.136585
4,2005,7588,1230,6.169106
5,2006,7246,1230,5.891057
6,2007,6847,1230,5.566667
7,2008,7165,1230,5.825203
8,2009,6987,1230,5.680488
9,2010,7369,1319,5.586808


Let's make two plots:
* A line plot showing the evolution of total goals per season(start).
* Bar chart with average number of goals per game.

In [10]:
fig = px.line(df_goals_season,
              x='season_start',
              y='total_goals',
              title='Number of goals per season',
              labels={
                  'total_goals': 'Number of goals',
                  'season_start': 'Season Start'
              })
fig.show()

In [11]:
fig = px.bar(df_goals_season,
             x='season_start',
             y='goals_per_game',
             range_y=[4, 6],
             title='Average number of goals per game',
             labels={
                 'total_goals': 'Average number of goals per game',
                 'season_start': 'Season Start'
             })
fig.show()

## Goals per team per season

In [12]:
query = """
SELECT
	SUBSTR(game_plays.game_id, 1, 4) AS season,
	team_info.teamName,
	COUNT(game_plays.event) AS number_of_goals,
	game_plays.team_id_for	
FROM
	game_plays
INNER JOIN
	team_info
		ON game_plays.team_id_for = team_info.team_id
WHERE
	event = 'Goal'
GROUP BY
	season, game_plays.team_id_for
ORDER BY
	season, number_of_goals DESC;
"""
df_teams_season = pd.read_sql(query, con)

In [13]:
df_teams_season.head()

,season,teamName,number_of_goals,team_id_for
0,2000,Devils,293,1
1,2000,Penguins,271,5
2,2000,Avalanche,263,21
3,2000,Senators,258,9
4,2000,Red Wings,253,17


Filtering a specific team:

In [14]:
df_teams_season[df_teams_season['teamName'] == 'Devils']

,season,teamName,number_of_goals,team_id_for
0,2000,Devils,293,1
50,2001,Devils,199,1
76,2002,Devils,205,1
101,2003,Devils,210,1
140,2005,Devils,243,1
173,2006,Devils,220,1
206,2007,Devils,209,1
223,2008,Devils,250,1
268,2009,Devils,213,1
299,2010,Devils,175,1


We can see the evolution of the number of goals of a specific team over the seasons. Let's explore the <font color='red'>Flames</font> evolution:

In [15]:
fig = px.line(df_teams_season[df_teams_season['teamName'] == 'Flames'],
              x='season',
              y='number_of_goals',
              title='Number of goals per season - Flames',
              labels={
                  'number_of_goals': 'Number of goals',
                  'season': 'Season Start'
              })
fig.show()

Let's put this into a function that returns the plotly figure object for a specific team.

In [16]:
def fig_team_goals(team, df=df_teams_season):
    return px.line(df[df['teamName'] == team],
                   x='season',
                   y='number_of_goals',
                   title=team + ' - Goals per season',
                   labels={
                       'number_of_goals': 'Number of goals',
                       'season': 'Season Start'
                   })

Let's test it for the Rangers:

In [17]:
fig_team_goals('Rangers').show()

Looking good! We'll go back to this later...

## Goals conceded per team per season

In [18]:
query = """
SELECT
	SUBSTR(game_plays.game_id, 1, 4) AS season,
	team_info.teamName,
	COUNT(game_plays.event) AS goals_conceded,
	game_plays.team_id_against
FROM
	game_plays
INNER JOIN
	team_info
		ON game_plays.team_id_against = team_info.team_id
WHERE
	event = 'Goal'
GROUP BY
	season, game_plays.team_id_against
ORDER BY
	season, goals_conceded DESC;
"""
df_teams_conceded = pd.read_sql(query, con)

In [19]:
df_teams_conceded.head()

,season,teamName,goals_conceded,team_id_against
0,2000,Rangers,283,3
1,2000,Thrashers,277,11
2,2000,Lightning,275,14
3,2000,Islanders,261,2
4,2000,Penguins,252,5


Similar to waht we did for the goals plot, we'll create a function to plot out the evolution of conceded goals:

In [20]:
def fig_team_conceded(team, df=df_teams_conceded):
    return px.line(df[df['teamName'] == team],
                   x='season',
                   y='goals_conceded',
                   title=team + ' - Goals Conceded per season',
                   labels={
                       'number_of_goals': 'Number of goals',
                       'season': 'Season Start'
                   })

In [21]:
fig_team_conceded('Rangers').show()

Now, it would be really nice to see this two curves (golas made and goals conceded) in a single plot. Let's see how we can do that using plotly's `go.Figure()` object:

In [22]:
fig = go.Figure()

fig.add_scatter(x = df_teams_season[df_teams_season['teamName'] == 'Devils']['season'],
                y = df_teams_season[df_teams_season['teamName'] == 'Devils']['number_of_goals'],
                name = 'Scored')

fig.add_scatter(x = df_teams_conceded[df_teams_conceded['teamName'] == 'Devils']['season'],
                y = df_teams_conceded[df_teams_conceded['teamName'] == 'Devils']['goals_conceded'],
                name = 'Conceded')

fig.update_layout(
    title="Goals - "+ 'Devils',
    xaxis_title="Season",
    yaxis_title="Goals"
)
fig.update_layout(hovermode="x unified")

Now that's way more informative (and pretty!). Let's get this one into a function too. We'll do two new things to the figure in the function:
* Rotate the x axes by 45 degrees to make it cleaner.
* Choose colors for the lines. Blue for scored goals and red for conceded ones.

In [23]:
def team_goals(team, df_pro=df_teams_season, df_con=df_teams_conceded):
    """Create plotly figure with line plots of goals scored and conceded
    for a given `team` across seasons"""

    fig = go.Figure()

    fig.add_scatter(x=df_pro[df_pro['teamName'] == team]['season'],
                    y=df_pro[df_pro['teamName'] == team]['number_of_goals'],
                    name='Scored',
                    line=dict(color="#0f3e66"))

    fig.add_scatter(x=df_con[df_con['teamName'] == team]['season'],
                    y=df_con[df_con['teamName'] == team]['goals_conceded'],
                    name='Conceded',
                    line=dict(color="#b53312"))

    fig.update_layout(title="Goals - " + team,
                      xaxis_title="Season",
                      yaxis_title="Goals")
    fig.update_layout(hovermode="x unified")
    fig.update_xaxes(tickangle=45)

    return fig

In [24]:
team_goals('Flames').show()

That looks great!

In [25]:
# df_teams_season

In [26]:
# df_teams_conceded

# Analyzing shot data

Our data contains (x,y) position of every game event. Let's make a quick function to filter out one season, one team and every **shot** event in that filter (these will be `Goal`, `Shot` and `Missed Shot`). Our function will further have an option to show the scatter plot for this events colored by the event type.

In [27]:
def shot_data(con, season, team_id, return_fig=True):

    query = """
    SELECT
        SUBSTR(game_id, 1, 4) AS season,
        play_id, game_id, team_id_for, team_id_against,
        event, secondaryType,
        x, y,
        period, periodType, periodTime, periodTimeRemaining,
        st_x, st_y,
        (periodTime + periodTimeRemaining) AS time_sum
    FROM
        game_plays
    WHERE
        event IN ('Goal', 'Shot', 'Missed Shot')
        AND
            (x <> 'NA' AND y <> 'NA')
        AND
            season = '{0}'
        AND
            team_id_for = {1}
    ORDER BY
        season, play_id;
    """
    
    df = pd.read_sql(query.format(season, team_id), con)
    
    df[["x", "y", "st_x", "st_y"]] = df[["x", "y", "st_x", "st_y"]].apply(pd.to_numeric)
    
    fig = None
    
    if return_fig:
    
        fig = px.scatter(df, x = 'st_x', y = 'st_y', color = 'event',
                         range_x = [-100, 100],
                         range_y = [-45, 45])

        # Set templates
        fig.update_layout(template="plotly_white")
    
    return df, fig

In [28]:
test_shot_data, fig = shot_data(con, 2014, 20)

In [29]:
fig.show()

In [30]:
test_shot_data

,season,play_id,game_id,team_id_for,team_id_against,event,secondaryType,x,y,period,periodType,periodTime,periodTimeRemaining,st_x,st_y,time_sum
0,2014,2014020003_114,2014020003,20,23,Shot,Snap Shot,72,8,2,REGULAR,41,1159,72,8,1200
1,2014,2014020003_13,2014020003,20,23,Shot,Snap Shot,-45,7,1,REGULAR,111,1089,45,-7,1200
2,2014,2014020003_135,2014020003,20,23,Goal,Wrist Shot,59,-11,2,REGULAR,248,952,59,-11,1200
3,2014,2014020003_148,2014020003,20,23,Shot,Slap Shot,37,-17,2,REGULAR,367,833,37,-17,1200
4,2014,2014020003_154,2014020003,20,23,Shot,Tip-In,75,-4,2,REGULAR,437,763,75,-4,1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3497,2014,2014030245_40,2014030245,20,24,Shot,Wrist Shot,-82,-26,1,REGULAR,287,913,82,26,1200
3498,2014,2014030245_48,2014030245,20,24,Missed Shot,NA,-46,32,1,REGULAR,333,867,46,-32,1200
3499,2014,2014030245_62,2014030245,20,24,Shot,Wrist Shot,-88,7,1,REGULAR,493,707,88,-7,1200
3500,2014,2014030245_76,2014030245,20,24,Goal,Slap Shot,-60,-16,1,REGULAR,643,557,60,16,1200


In [31]:
test_shot_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3502 entries, 0 to 3501
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   season               3502 non-null   object
 1   play_id              3502 non-null   object
 2   game_id              3502 non-null   int64 
 3   team_id_for          3502 non-null   object
 4   team_id_against      3502 non-null   object
 5   event                3502 non-null   object
 6   secondaryType        3502 non-null   object
 7   x                    3502 non-null   int64 
 8   y                    3502 non-null   int64 
 9   period               3502 non-null   int64 
 10  periodType           3502 non-null   object
 11  periodTime           3502 non-null   int64 
 12  periodTimeRemaining  3502 non-null   int64 
 13  st_x                 3502 non-null   int64 
 14  st_y                 3502 non-null   int64 
 15  time_sum             3502 non-null   int64 
dtypes: int

In [32]:
# fig = px.scatter(test_shot_data, x = 'x', y = 'y', color = 'event')
# fig.show()

In [33]:
# fig = px.scatter(test_shot_data, x = 'x', y = 'y', facet_row = 'event')
# fig.show()

In [34]:
fig = px.scatter(test_shot_data, x = 'st_x', y = 'st_y', color = 'event',
                 range_x = [-100, 100],
                 range_y = [-45, 45])

# Set templates
fig.update_layout(template="plotly_white")

fig.show()

We can use the `facet_row` argument to gat a different plot for each possible period of the game:

In [35]:
fig = px.scatter(test_shot_data, x = 'st_x', y = 'st_y', color = 'event',
                 range_x = [-100, 100],
                 range_y = [-45, 45],
                 facet_row = 'period',
                 height=1200)

fig.show()

In [36]:
# plt.figure(figsize=(15, 6.375))

# x = test_shot_data['st_x'].values
# y = test_shot_data['st_y'].values

# plt.hexbin(x, y, gridsize=30, mincnt=1, edgecolors="face", cmap="viridis")
# # plt.scatter(x, y, s=1, c="red")

# plt.xlim(-100, 100)
# plt.ylim(-45, 45)
# plt.show()

In [37]:
# test_shot_data.head()

In [38]:
import base64

Now, to make things a bit more visual, we can set a rink image as a background and see where shots were taken. We need to play around with the figure size to fit official rink size, but plotly has the flexibility for it.

Get official rink size [here](https://en.wikipedia.org/wiki/Ice_hockey_rink).

We'll do a heatmap of the shots to identify regions of higher concentration of shots. We could do this in a similar way for every kind of shot type.

In [39]:
IMAGE_FILENAME1 = './images/NHL-rink-white.jpg'
image1 = base64.b64encode(open(IMAGE_FILENAME1, 'rb').read())

fig = px.density_heatmap(test_shot_data,
                         x="st_x",
                         y="st_y",
                         nbinsx=50,
                         nbinsy=50,
                         range_x=[-100, 100],
                         range_y=[-45, 45],
                         color_continuous_scale="Reds",
#                          color_continuous_scale=px.colors.diverging.Picnic,
#                          color_continuous_midpoint=0,
                         )

fig.update_traces(opacity=0.5)

fig.add_layout_image(dict(source='data:image/jpg;base64,{}'.format(image1.decode()),
                          xref="x",
                          yref="y",
                          x=-100, y=42.5,
                          sizex=200,
                          sizey=85, 
                          sizing="stretch",
                          opacity=1,
                          layer="below"))
# # Rectangle marking size
# fig.add_shape(type="rect",
#     x0=-100, y0=-42.5, x1=100, y1=42.5,
#     line=dict(color="RoyalBlue"),
# )

fig.update_layout(template="simple_white")

fig.show()

Let's get a quick reference dictionary to match team_id with team name:

In [40]:
# team id to team name
query = """
SELECT
	team_id, teamName
FROM
	team_info
ORDER BY
	team_id ASC;
"""
df_teams = pd.read_sql(query, con)

In [41]:
df_teams

,team_id,teamName
0,1,Devils
1,2,Islanders
2,3,Rangers
3,4,Flyers
4,5,Penguins
5,6,Bruins
6,7,Sabres
7,8,Canadiens
8,9,Senators
9,10,Maple Leafs


In [42]:
# team_dict = df_teams.set_index('team_id').T.to_dict('list')
team_dict = dict(df_teams.values)

In [43]:
team_dict

{1: 'Devils',
 2: 'Islanders',
 3: 'Rangers',
 4: 'Flyers',
 5: 'Penguins',
 6: 'Bruins',
 7: 'Sabres',
 8: 'Canadiens',
 9: 'Senators',
 10: 'Maple Leafs',
 11: 'Thrashers',
 12: 'Hurricanes',
 13: 'Panthers',
 14: 'Lightning',
 15: 'Capitals',
 16: 'Blackhawks',
 17: 'Red Wings',
 18: 'Predators',
 19: 'Blues',
 20: 'Flames',
 21: 'Avalanche',
 22: 'Oilers',
 23: 'Canucks',
 24: 'Ducks',
 25: 'Stars',
 26: 'Kings',
 27: 'Coyotes',
 28: 'Sharks',
 29: 'Blue Jackets',
 30: 'Wild',
 52: 'Jets',
 53: 'Coyotes',
 54: 'Golden Knights'}

With this dict we can get the team name on the plot title:

In [44]:
def plot_heatmap(con, season, team_id, event):
    
    df, f = shot_data(con, season, team_id, return_fig=False)

    fig = px.density_heatmap(df.query(f"event == '{event}'"),
                             x="st_x",
                             y="st_y",
                             nbinsx=50,
                             nbinsy=50,
                             range_x=[-100, 100],
                             range_y=[-45, 45],
                             color_continuous_scale="Reds",
                             title=team_dict[team_id]+' '+str(season)+' '+event+'s'
                             )

    fig.update_traces(opacity=0.6)

    fig.add_layout_image(dict(source='data:image/jpg;base64,{}'.format(image1.decode()),
                              xref="x",
                              yref="y",
                              x=-100, y=42.5,
                              sizex=200,
                              sizey=85, 
                              sizing="stretch",
                              opacity=1,
                              layer="below"))

    fig.update_layout(template="simple_white")
    
    #legend
    fig.update_layout(showlegend=False)

    #x axis
    fig.update_xaxes(visible=False)

    #y axis    
    fig.update_yaxes(visible=False)
    
    return fig

In [45]:
fig = plot_heatmap(con, 2014, 20, 'Goal')
fig.show()

In [46]:
fig = plot_heatmap(con, 2014, 20, 'Shot')
fig.show()

In [47]:
fig = plot_heatmap(con, 2014, 1, 'Shot')
fig.show()

In [48]:
fig = plot_heatmap(con, 2014, 20, 'Missed Shot')
fig.show()

Let's get an idea on the number of shot events:

In [49]:
test_shot_data['event'].value_counts()

Shot           2232
Missed Shot    1004
Goal            266
Name: event, dtype: int64

In [50]:
test_shot_data.query("event == 'Shot'")

,season,play_id,game_id,team_id_for,team_id_against,event,secondaryType,x,y,period,periodType,periodTime,periodTimeRemaining,st_x,st_y,time_sum
0,2014,2014020003_114,2014020003,20,23,Shot,Snap Shot,72,8,2,REGULAR,41,1159,72,8,1200
1,2014,2014020003_13,2014020003,20,23,Shot,Snap Shot,-45,7,1,REGULAR,111,1089,45,-7,1200
3,2014,2014020003_148,2014020003,20,23,Shot,Slap Shot,37,-17,2,REGULAR,367,833,37,-17,1200
4,2014,2014020003_154,2014020003,20,23,Shot,Tip-In,75,-4,2,REGULAR,437,763,75,-4,1200
6,2014,2014020003_180,2014020003,20,23,Shot,Wrist Shot,43,14,2,REGULAR,735,465,43,14,1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3494,2014,2014030245_338,2014030245,20,24,Shot,Backhand,-88,8,3,REGULAR,825,375,88,-8,1200
3496,2014,2014030245_372,2014030245,20,24,Shot,Backhand,-77,9,3,REGULAR,1199,1,77,-9,1200
3497,2014,2014030245_40,2014030245,20,24,Shot,Wrist Shot,-82,-26,1,REGULAR,287,913,82,26,1200
3499,2014,2014030245_62,2014030245,20,24,Shot,Wrist Shot,-88,7,1,REGULAR,493,707,88,-7,1200


Choose color scale [here](https://plotly.com/python/builtin-colorscales/).

In [51]:
test_shot_data

,season,play_id,game_id,team_id_for,team_id_against,event,secondaryType,x,y,period,periodType,periodTime,periodTimeRemaining,st_x,st_y,time_sum
0,2014,2014020003_114,2014020003,20,23,Shot,Snap Shot,72,8,2,REGULAR,41,1159,72,8,1200
1,2014,2014020003_13,2014020003,20,23,Shot,Snap Shot,-45,7,1,REGULAR,111,1089,45,-7,1200
2,2014,2014020003_135,2014020003,20,23,Goal,Wrist Shot,59,-11,2,REGULAR,248,952,59,-11,1200
3,2014,2014020003_148,2014020003,20,23,Shot,Slap Shot,37,-17,2,REGULAR,367,833,37,-17,1200
4,2014,2014020003_154,2014020003,20,23,Shot,Tip-In,75,-4,2,REGULAR,437,763,75,-4,1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3497,2014,2014030245_40,2014030245,20,24,Shot,Wrist Shot,-82,-26,1,REGULAR,287,913,82,26,1200
3498,2014,2014030245_48,2014030245,20,24,Missed Shot,NA,-46,32,1,REGULAR,333,867,46,-32,1200
3499,2014,2014030245_62,2014030245,20,24,Shot,Wrist Shot,-88,7,1,REGULAR,493,707,88,-7,1200
3500,2014,2014030245_76,2014030245,20,24,Goal,Slap Shot,-60,-16,1,REGULAR,643,557,60,16,1200


Now, instead of the heatmap, we can get a function to plot the position of shots made and mark goals and *shots* (not scored).

We'll try to write a general function that does this but also allows for plotting a single game, given the game_id).

In [52]:
def plot_shot_type(con, season, team_id, shot_type, game_id = None):
    """
    Plots shot position with background rink (NHL official size).
    Arguments:
    - con: conncetion to nhl database (given in project folder or converted from kaggle dataset)
    - season: integer value of start year of season (currently available: 2000 to 2019)
    - team_id: ID of team as given by the table team_info
    - shot_type: secondary event type of shot events. 
        * Available: 'Wrist Shot', 'Slap Shot', 'Snap Shot', 'Backhand', 'Tip-In', 'Deflected', 'Wrap-around'.
    - game_id (optional): if None is given, plots the entire season. Otherwise, plots only shots for specific game_id.
    """

    df, f = shot_data(con, season, team_id, return_fig=False)
    
    if game_id:
        df = df.query(f"secondaryType == '{shot_type}' and game_id == {game_id}")
        title = team_dict[team_id] + ' ' + str(season) + ' Game ID: '+ str(game_id) + ' ' + shot_type +\
                f's <br><sup>{len(df)} shots</sup>'
    
    else:
        df = df.query(f"secondaryType == '{shot_type}'")
        title = team_dict[team_id] + ' ' + str(season) + ' ' + shot_type +\
                f's <br><sup>{len(df)} shots</sup>'

    number_of_shots = len(df)

    marker_size = 12
    marker_width = 1

    fig = px.scatter(
        df,
        x='st_x',
        y='st_y',
        color='event',
        symbol='event',
        range_x=[-100, 100],
        range_y=[-45, 45],
        title=title,
        color_discrete_map={  # replaces default color mapping by value
            "Goal": "DarkRed",
            "Shot": "LawnGreen"
        },
        symbol_map={  # replaces default symbol mapping by value
            "Shot": "x",
            "Goal": "circle"
        })

    fig.update_traces(marker=dict(size=marker_size,
                                  line=dict(width=marker_width,
                                            color='DarkSlateGrey')),
                      selector=dict(mode='markers'),
                      opacity=0.6)

    fig.add_layout_image(
        dict(source='data:image/jpg;base64,{}'.format(image1.decode()),
             xref="x",
             yref="y",
             x=-100,
             y=42.5,
             sizex=200,
             sizey=85,
             sizing="stretch",
             opacity=0.6,
             layer="below"))

    #x axis
    fig.update_xaxes(visible=False)

    #y axis
    fig.update_yaxes(visible=False)

    # Set templates
    fig.update_layout(template="plotly_white")

    return fig

In [53]:
fig = plot_shot_type(con, 2014, 20, 'Slap Shot')
fig.show()

In [54]:
fig = plot_shot_type(con, 2014, 20, 'Wrist Shot')
fig.show()

In [55]:
fig = plot_shot_type(con, 2014, 20, 'Snap Shot')
fig.show()

In [56]:
fig = plot_shot_type(con, 2014, 2, 'Backhand')
fig.show()

In [57]:
# Testing over 1000 shots
shot_type = 'Wrist Shot'
season = 2014
team_id = 20

df, f = shot_data(con, season, team_id, return_fig=False)

df = df.query(f"secondaryType == '{shot_type}'")

number_of_shots = len(df)

marker_size = 12
marker_width = 1

fig = px.scatter(
    df,
    x='st_x',
    y='st_y',
    color='event',
    symbol='event',
    range_x=[-100, 100],
    range_y=[-45, 45],
    title=team_dict[team_id] + ' ' + str(season) + ' ' + shot_type +
    f's <br><sup>{len(df)} shots</sup>',
#     color_discrete_map={  # replaces default color mapping by value
#         "Shot": "DarkRed",
#         "Goal": "LawnGreen"
#     },
    symbol_map={  # replaces default symbol mapping by value
        "Shot": "x",
        "Goal": "circle"
    })

fig.update_traces(marker=dict(size=marker_size,
                              line=dict(width=marker_width,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'),
                  opacity=0.4)

fig.add_layout_image(
    dict(source='data:image/jpg;base64,{}'.format(image1.decode()),
         xref="x",
         yref="y",
         x=-100,
         y=42.5,
         sizex=200,
         sizey=85,
         sizing="stretch",
         opacity=0.6,
         layer="below"))

#x axis
fig.update_xaxes(visible=False)

#y axis
fig.update_yaxes(visible=False)

# Set templates
fig.update_layout(template="plotly_white")

fig.show()

In [58]:
test_shot_data['secondaryType'].value_counts()

Wrist Shot     1310
NA             1004
Slap Shot       441
Snap Shot       366
Backhand        187
Tip-In          139
Deflected        32
Wrap-around      23
Name: secondaryType, dtype: int64

In [59]:
flames_2014_shots, fig = shot_data(con,2014,20)

In [60]:
flames_2014_shots

,season,play_id,game_id,team_id_for,team_id_against,event,secondaryType,x,y,period,periodType,periodTime,periodTimeRemaining,st_x,st_y,time_sum
0,2014,2014020003_114,2014020003,20,23,Shot,Snap Shot,72,8,2,REGULAR,41,1159,72,8,1200
1,2014,2014020003_13,2014020003,20,23,Shot,Snap Shot,-45,7,1,REGULAR,111,1089,45,-7,1200
2,2014,2014020003_135,2014020003,20,23,Goal,Wrist Shot,59,-11,2,REGULAR,248,952,59,-11,1200
3,2014,2014020003_148,2014020003,20,23,Shot,Slap Shot,37,-17,2,REGULAR,367,833,37,-17,1200
4,2014,2014020003_154,2014020003,20,23,Shot,Tip-In,75,-4,2,REGULAR,437,763,75,-4,1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3497,2014,2014030245_40,2014030245,20,24,Shot,Wrist Shot,-82,-26,1,REGULAR,287,913,82,26,1200
3498,2014,2014030245_48,2014030245,20,24,Missed Shot,NA,-46,32,1,REGULAR,333,867,46,-32,1200
3499,2014,2014030245_62,2014030245,20,24,Shot,Wrist Shot,-88,7,1,REGULAR,493,707,88,-7,1200
3500,2014,2014030245_76,2014030245,20,24,Goal,Slap Shot,-60,-16,1,REGULAR,643,557,60,16,1200


# Individual games

From [these](https://gitlab.com/dword4/nhlapi/-/blob/master/stats-api.md#game-ids) docs:

> Game IDs:
The first 4 digits identify the season of the game (ie. 2017 for the 2017-2018 season). The next 2 digits give the type of game, where 01 = preseason, 02 = regular season, 03 = playoffs, 04 = all-star. The final 4 digits identify the specific game number. For regular season and preseason games, this ranges from 0001 to the number of games played. (1271 for seasons with 31 teams (2017 and onwards) and 1230 for seasons with 30 teams). For playoff games, the 2nd digit of the specific number gives the round of the playoffs, the 3rd digit specifies the matchup, and the 4th digit specifies the game (out of 7).

In [61]:
flames_2014_shots.sort_values(by='game_id')

,season,play_id,game_id,team_id_for,team_id_against,event,secondaryType,x,y,period,periodType,periodTime,periodTimeRemaining,st_x,st_y,time_sum
0,2014,2014020003_114,2014020003,20,23,Shot,Snap Shot,72,8,2,REGULAR,41,1159,72,8,1200
31,2014,2014020003_86,2014020003,20,23,Shot,Slap Shot,-55,-27,1,REGULAR,936,264,55,27,1200
30,2014,2014020003_82,2014020003,20,23,Shot,Wrist Shot,-43,13,1,REGULAR,864,336,43,-13,1200
29,2014,2014020003_81,2014020003,20,23,Missed Shot,NA,-36,-30,1,REGULAR,861,339,36,30,1200
28,2014,2014020003_6,2014020003,20,23,Missed Shot,NA,-60,11,1,REGULAR,29,1171,60,-11,1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3472,2014,2014030245_107,2014030245,20,24,Shot,Tip-In,-81,-5,1,REGULAR,981,219,81,5,1200
3471,2014,2014030245_10,2014030245,20,24,Shot,Wrist Shot,-61,-17,1,REGULAR,38,1162,61,17,1200
3500,2014,2014030245_76,2014030245,20,24,Goal,Slap Shot,-60,-16,1,REGULAR,643,557,60,16,1200
3485,2014,2014030245_238,2014030245,20,24,Shot,Wrist Shot,80,-16,2,REGULAR,1087,113,80,-16,1200


In [62]:
flames_2014_shots['game_id'].nunique()

91

In [63]:
fig = plot_shot_type(con, season=2014, team_id=20, shot_type='Wrist Shot', game_id=2014020003)
fig.show()